Importing liabraries

In [1]:
!pip install keras
!pip install tensorflow
#Install related libraries for the course.
#This is a common requirement for all other exampels too

!pip install pandas
!pip install tensorflow
!pip install sklearn
!pip install matplotlib
!pip install comet_ml

In [1]:
# import comet_ml at the top of your file
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(log_code=True,
    api_key="KgERfs0s0QEiRrMhOJYwGn5pk",
    project_name="dynamic-malware-analysis",
    workspace="thevindus",
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in 'C:\\Users\\DELL\\LSTM3' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/thevindus/dynamic-malware-analysis/9dc5f9c0dbb6470ea4ee286261a4dbac



In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
#import autoreload
#?autoreload


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pickle as pickle

import random
random.seed(42)


from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score

from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.layers import Dense
import tensorflow as tf
print(tf.__version__)

2.12.0


Loading Dataset

In [4]:
malware_API_data = pd.read_excel("malware_API_dataset.xlsx", header = None, dtype= str)

In [5]:
dataset = pd.concat([malware_API_data], ignore_index=True)

In [6]:
from sklearn.utils import shuffle
dataset = shuffle(dataset)


In [7]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,588,589,590,591,592,593,594,595,596,597
15923,Trojan,Trojan-Spy.Win32.Zbot.lcdm,06eb604bc250d7579fe6c7e7cb5490b44cb078cafe38cd...,CreateEventW,CreateEventA,LoadLibraryExW,LoadLibraryExA,LoadLibraryA,IsDBCSLeadByte,FindResourceExW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11229,Trojan,Trojan-FakeAV.Win32.Agent.dlw,39de5f0e64e3e25c7fe00750fa4809ad67e53c2c38617b...,IsDebuggerPresent,CreateWaitableTimerW,CreateWaitableTimerA,SearchPathW,CreateFileW,CreateFileMappingW,MapViewOfFileEx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953,Trojan,HEUR:Trojan.Win32.Generic,048bcfe716f763bc96a25c181bb8221116284d01a99aa9...,LocalAlloc,CreateSemaphoreW,CreateSemaphoreA,CreateEventA,CreateFileW,DeviceIoControl,DuplicateHandle,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11702,Trojan,Trojan-FakeAV.Win32.Agent.dov,dc32dbfa9c7f0824b6a271538366942f3a3683dd4d16d4...,FindResourceExW,LoadResource,GetProcessWindowStation,GetUserObjectInformationW,lstrcpyW,CharPrevW,lstrcpynW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7637,Trojan,Packed.Win32.Krap.an,1289464ca076ff070b2076ff114cb792e9e4fe205cebc6...,CreateEventW,CreateSemaphoreW,RegisterClipboardFormatA,GetSystemDirectoryA,GetVersionExW,IsDBCSLeadByte,CompareStringW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6967,Trojan,not-a-virus:Downloader.Win32.LMN.it,15189210f4cbdf562a193e16bc675002135fa58f0649e3...,VirtualQueryEx,GetModuleFileNameW,GetModuleFileNameA,CreateSemaphoreW,CreateSemaphoreA,GetCommandLineA,GetStartupInfoA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16811,Worm,Worm.Win32.AutoRun.hjk,05ff97c14e1548de650a99d2189118d31dacaa6911fad6...,GetVersionExW,GetVersionExA,GetModuleHandleW,GetModuleHandleA,GetProcAddress,LocalAlloc,lstrlenW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
383,Backdoor,Backdoor.Win32.Shiz.cebw,10a33e58e250af3fbe0d523f699bc407649fc0775b050a...,GetVersionExW,GetVersionExA,GetModuleHandleW,GetModuleHandleA,GetProcAddress,LocalAlloc,lstrlenW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13542,Trojan,Trojan-FakeAV.Win32.SmartFortress2012.jv,8a88b6ce08b60afa59cdb1b5979ba027eea991682acf72...,GetDeviceCaps,ReleaseDC,CreateMailslotW,lstrlenA,SearchPathW,CreateFileW,CreateFileMappingW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print(dataset)

            0                                         1    \
15923    Trojan                Trojan-Spy.Win32.Zbot.lcdm   
11229    Trojan             Trojan-FakeAV.Win32.Agent.dlw   
1953     Trojan                 HEUR:Trojan.Win32.Generic   
11702    Trojan             Trojan-FakeAV.Win32.Agent.dov   
7637     Trojan                      Packed.Win32.Krap.an   
...         ...                                       ...   
6967     Trojan       not-a-virus:Downloader.Win32.LMN.it   
16811      Worm                    Worm.Win32.AutoRun.hjk   
383    Backdoor                  Backdoor.Win32.Shiz.cebw   
13542    Trojan  Trojan-FakeAV.Win32.SmartFortress2012.jv   
2173     Trojan                 HEUR:Trojan.Win32.Generic   

                                                     2    \
15923  06eb604bc250d7579fe6c7e7cb5490b44cb078cafe38cd...   
11229  39de5f0e64e3e25c7fe00750fa4809ad67e53c2c38617b...   
1953   048bcfe716f763bc96a25c181bb8221116284d01a99aa9...   
11702  dc32dbfa9c7f0824b6a2

In [9]:
dataset_X = dataset.values[:,3:598]
dataset_X

array([['CreateEventW', 'CreateEventA', 'LoadLibraryExW', ..., nan, nan,
        nan],
       ['IsDebuggerPresent', 'CreateWaitableTimerW',
        'CreateWaitableTimerA', ..., nan, nan, nan],
       ['LocalAlloc', 'CreateSemaphoreW', 'CreateSemaphoreA', ..., nan,
        nan, nan],
       ...,
       ['GetVersionExW', 'GetVersionExA', 'GetModuleHandleW', ..., nan,
        nan, nan],
       ['GetDeviceCaps', 'ReleaseDC', 'CreateMailslotW', ..., nan, nan,
        nan],
       ['WaitForSingleObjectEx', 'WaitForSingleObject', 'SearchPathW',
        ..., nan, nan, nan]], dtype=object)

In [10]:
dataset_Y = dataset.values[:,0]
dataset_Y
#dataset.values[5000,0]

array(['Trojan', 'Trojan', 'Trojan', ..., 'Backdoor', 'Trojan', 'Trojan'],
      dtype=object)

Preprocessing data

In [11]:
le = preprocessing.LabelEncoder()

dataset_X_pandas = pd.DataFrame(dataset_X,dtype=str)
dataset_X_pandas.apply(le.fit_transform)

dataset_X = dataset_X_pandas.apply(le.fit_transform).values[:,:]
dataset_X

array([[  4,   6, 104, ...,  25,  15,  10],
       [ 79,  18,  23, ...,  25,  15,  10],
       [ 85,  16,  20, ...,  25,  15,  10],
       ...,
       [ 63,  82,  61, ...,  25,  15,  10],
       [ 29, 119,  15, ...,  25,  15,  10],
       [119, 138, 123, ...,  25,  15,  10]])

In [12]:
le = preprocessing.LabelEncoder()

dataset_Y_pandas = pd.DataFrame(dataset_Y,dtype=str)
dataset_Y_pandas.apply(le.fit_transform)

dataset_Y = dataset_Y_pandas.apply(le.fit_transform).values[:,:]

dataset_Y

array([[6],
       [6],
       [6],
       ...,
       [2],
       [6],
       [6]])

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(dataset_X, dataset_Y, test_size=0.5, random_state=42)

In [14]:
from sklearn.preprocessing import MinMaxScaler

data = X_train
scaler = MinMaxScaler()
print(scaler.fit(data))

print(scaler.transform(data))

X_train = scaler.transform(data)

X_test = scaler.transform(X_test)

MinMaxScaler()
[[0.4015748  0.56164384 0.60927152 ... 1.         1.         1.        ]
 [0.65354331 0.84246575 0.70198675 ... 1.         1.         1.        ]
 [0.17322835 0.41780822 0.71523179 ... 1.         1.         1.        ]
 ...
 [0.97637795 0.51369863 0.97350993 ... 1.         1.         1.        ]
 [0.97637795 0.51369863 0.97350993 ... 1.         1.         1.        ]
 [0.17322835 0.41780822 0.71523179 ... 1.         1.         1.        ]]


In [15]:
from sklearn.preprocessing import MinMaxScaler

data = Y_train
scaler = MinMaxScaler()
print(scaler.fit(data))

print(scaler.transform(data))

Y_train = scaler.transform(data)
Y_test = scaler.transform(Y_test)

MinMaxScaler()
[[0.        ]
 [0.36363636]
 [0.54545455]
 ...
 [0.54545455]
 [0.54545455]
 [0.54545455]]


In [16]:
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(Y_train,num_classes=12)  # Assuming Y
Y_test = to_categorical(Y_test,num_classes=12)  #

In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

In [18]:
vocab_size = 1000
max_sequence_length = 595

In [19]:
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=max_sequence_length))
model.add(LSTM(64))
model.add(Dense(12, activation='softmax'))
model.summary()

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 595, 32)           32000     
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 12)                780       
                                                                 
Total params: 57,612
Trainable params: 57,612
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, Y_train, batch_size=32, epochs=100,validation_split=0.2)

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/100
216/216 [==============================] - 71s 319ms/step - loss: 0.1699 - accuracy: 0.9945 - val_loss: 0.0053 - val_accuracy: 0.9994
Epoch 2/100
216/216 [==============================] - 69s 318ms/step - loss: 0.0074 - accuracy: 0.9991 - val_loss: 0.0051 - val_accuracy: 0.9994
Epoch 3/100
216/216 [==============================] - 66s 304ms/step - loss: 0.0073 - accuracy: 0.9991 - val_loss: 0.0050 - val_accuracy: 0.9994
Epoch 4/100
216/216 [==============================] - 68s 314ms/step - loss: 0.0073 - accuracy: 0.9991 - val_loss: 0.0051 - val_accuracy: 0.9994
Epoch 5/100
216/216 [==============================] - 69s 319ms/step - loss: 0.0073 - accuracy: 0.9991 - val_loss: 0.0050 - val_accuracy: 0.9994
Epoch 6/100
216/216 [==============================] - 22295s 104s/step - loss: 0.0073 - accuracy: 0.9991 - val_loss: 0.0050 - val_accuracy: 0.9994
Epoch 7/100
216/216 [==============================] - 90s 416ms/step - loss: 0.0072 - accuracy: 0.9991 - val_loss: 0.0049